# homework1

In [3]:
import os
from langchain_community.document_loaders import PDFMinerLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_community.chat_models import ChatTongyi
import requests

In [ ]:
os.environ["DASHSCOPE_API_KEY"] = "" 

# 下载PDF文件
pdf_url = " https://storage.googleapis.com/deepmind-media/Era-of-Experience%20/The%20Era%20of%20Experience%20Paper.pdf"
local_path = "era_of_experience.pdf"
with open(local_path, "wb") as f:
    f.write(requests.get(pdf_url).content)

docs = PDFMinerLoader(local_path).load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)

embeddings = DashScopeEmbeddings(
    model="text-embedding-v1",  
    dashscope_api_key=os.environ["DASHSCOPE_API_KEY"]
)
vectorstore = FAISS.from_documents(documents=chunks, embedding=embeddings)

qa_chain = RetrievalQA.from_chain_type(
    llm=ChatTongyi(
        model_name="qwen-turbo",  
        dashscope_api_key=os.environ["DASHSCOPE_API_KEY"]
    ),
    retriever=vectorstore.as_retriever(),
    chain_type="stuff"
)

In [5]:
# 示例查询
query = "这篇论文的主要观点是什么？"
result = qa_chain.invoke({"query": query})
print(result["result"])

这篇论文的主要观点是，人工智能（AI）通过从与世界的互动中学习（即“经验”），可以实现突破性的进展，并可能发展成真正超越人类水平的智能代理。然而，这种基于经验的学习方法也带来了新的安全风险和挑战，需要进一步研究以确保安全过渡到这一新时代。同时，论文强调了使用“基于环境的奖励”来指导AI学习的重要性，而不是仅依赖人类的主观判断。
